## 수치 미분을 이용한 심층 신경망 학습

## Import modules

In [1]:
import time
import numpy as np

## 유틸리티 함수

In [2]:
epsilon = 0.0001

def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def mean_squared_error(h, y):
    return 1 / 2 * np.mean(np.square(h - y))

## Dense Layer 구현

In [7]:
class Dense:
    def __init__(self, W, b, a):
        self.W = W
        self.b = b
        self.a = a

        self.dW = np.zeros_like(self.W) # W와 같은 모양의 0 numpy
        self.db = np.zeros_like(self.b)
    
    def __call__(self, x):
        return self.a(_m(_t(self.W), x)+self.b) # matmul(ixo, ix1) + ox1

## 심층신경망 구현

In [8]:
class DNN:
    def __init__(self, hidden_depth, num_neuron, num_input, num_output, activation=sigmoid):
        def init_var(i, o):
            return np.random.normal(0.0, 0.01, (i, o)), np.zeros((o,))

        self.sequence = list()
        # First hidden layer
        W, b = init_var(num_input, num_neuron)
        self.sequence.append(Dense(W, b, activation))
        
        # Hidden layers
        for _ in range(hidden_depth-1):
            W, b = init_var(num_neuron, num_neuron)
            self.sequence.append(Dense(W, b, activation))

        # Output layer
        W, b = init_var(num_neuron, num_output)
        self.sequence.append(Dense(W, b, activation))

    def __call__(self, x):
        for layer in self.sequence:
            x = layer(x)
        return x

    def calc_gradient(self, x, y, loss_func):
        
        def get_new_sequence(layer_index, new_layer):
            new_sequence = list()
            for i, layer in enumerate(self.sequence):
                if i == layer_index:
                    new_sequence.append(new_layer)
                else:
                    new_sequence.append(layer)
            return new_sequence

        def eval_sequence(x, sequence):
            for layer in sequence:
                x = layer(x)
            return x
        
        loss = loss_func(self(x), y)
        # scalar의 값을 조금 씩 바꿔서 Numerical Gradient 구현
        for layer_id, layer in enumerate(self.sequence):
            for w_i, w in enumerate(layer.W):   # 아직은 벡터
                for w_j, ww in enumerate(w):
                    W = np.copy(layer.W)
                    W[w_i][w_j] = ww + epsilon
                    
                    new_layer = Dense(W, layer.b, layer.a)
                    new_seq = get_new_sequence(layer_id, new_layer)
                    h = eval_sequence(x, new_seq)
                    
                    # (f(x+eps) - f(x)) / eps
                    num_grad = (loss_func(h,y)-loss)/epsilon
                    layer.dW[w_i][w_j] = num_grad
            for b_i, bb in enumerate(layer.b):
                b = np.copy(layer.b)
                b[b_i] = bb+epsilon

                new_layer = Dense(layer.W, b, layer.a)
                new_seq = get_new_sequence(layer_id, new_layer)
                h = eval_sequence(x, new_seq)
                
                num_grad = (loss_func(h,y)-loss)/epsilon
                layer.db[b_i] = num_grad
        return loss


## 경사하강 학습법

In [9]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    loss = network.calc_gradient(x, y, loss_obj)
    for layer in network.sequence:
        layer.W += -alpha * layer.dW
        layer.b += -alpha * layer.db
    return loss

## 동작 테스트

In [12]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

dnn = DNN(hidden_depth=10, num_neuron=32, num_input=10, num_output=2, activation=sigmoid)

t = time.time()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, mean_squared_error, 0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

Epoch 0: Test loss 0.4259118609353312
Epoch 1: Test loss 0.4235372729294312
Epoch 2: Test loss 0.4211757413827673
Epoch 3: Test loss 0.4188274312191471
Epoch 4: Test loss 0.416492500507075
Epoch 5: Test loss 0.4141711004752838
Epoch 6: Test loss 0.4118633755357364
Epoch 7: Test loss 0.4095694633132235
Epoch 8: Test loss 0.4072894946816411
Epoch 9: Test loss 0.4050235938068666
Epoch 10: Test loss 0.40277187819553006
Epoch 11: Test loss 0.4005344587498181
Epoch 12: Test loss 0.39831143982797557
Epoch 13: Test loss 0.3961029193101566
Epoch 14: Test loss 0.39390898866935703
Epoch 15: Test loss 0.391729733047352
Epoch 16: Test loss 0.3895652313352048
Epoch 17: Test loss 0.3874155562580749
Epoch 18: Test loss 0.3852807744641048
Epoch 19: Test loss 0.3831609466172481
Epoch 20: Test loss 0.3810561274936154
Epoch 21: Test loss 0.37896636608103024
Epoch 22: Test loss 0.3768917056817879
Epoch 23: Test loss 0.37483218401804064
Epoch 24: Test loss 0.37278783333989707
Epoch 25: Test loss 0.370758680